In [12]:
from sklearn.datasets import make_moons
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import sys
import mpl_toolkits
sys.setrecursionlimit(1000000)
from mpl_toolkits.basemap import Basemap

In [13]:
#%% md


#%%

%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.geoaxes import GeoAxesSubplot

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import fetch_species_distributions, load_digits
from sklearn.model_selection import GridSearchCV, LeaveOneOut, train_test_split
from sklearn.neighbors import KernelDensity

sns.set()
plt.rc('font', family='SimHei')
plt.rc('axes', unicode_minus=False)

#%%

x_train = np.hstack((np.random.normal(2, 1.66, 200), np.random.normal(8, 2.33, 200)))

model = KernelDensity(bandwidth=1.0, kernel='gaussian')
model.fit(x_train[:, np.newaxis])
x_range = np.linspace(x_train.min() - 1, x_train.max() + 1, 500)
x_log_prob = model.score_samples(x_range[:, np.newaxis])  # 这个东西返回概率的对数
x_prob = np.exp(x_log_prob)

print(x_range.shape)
print(x_log_prob.shape)

plt.figure(figsize=(10, 10))
r = plt.hist(
    x=x_train,
    bins=50,
    density=True,
    histtype='stepfilled',
    color='red',
    alpha=0.5,
)
plt.fill_between(
    x=x_range,
    y1=x_prob,
    y2=0,
    color='green',
    alpha=0.5,
    label='KDE',
)
plt.plot(x_range, x_prob, color='gray')
plt.vlines(x=2, ymin=0, ymax=r[0].max() + 0.01, color='k', linestyle='--', alpha=0.7)
plt.vlines(x=8, ymin=0, ymax=r[0].max() + 0.01, color='k', linestyle='--', alpha=0.7)
plt.ylim(0, r[0].max() + 0.011)
plt.legend(loc='upper right')


# #%%

x_train = np.hstack((np.random.normal(2, 1.66, 200), np.random.normal(8, 2.33, 200)))

grid = GridSearchCV(
    estimator=KernelDensity(kernel='gaussian'),
    param_grid={
    'bandwidth': 10 ** np.linspace(-1, 1, 100)},
    cv=LeaveOneOut(),
)
grid.fit(x_train[:, np.newaxis])
print(f'：{grid.best_params_["bandwidth"]}')



def construct_grids(batch): 
    """Construct the map grid from the batch object

    Parameters
    ----------
    batch : Batch object
        The object returned by :func:`fetch_species_distributions`

    Returns
    -------
    (xgrid, ygrid) : 1-D arrays
        The grid corresponding to the values in batch.coverages
    """
    # x,y coordinates for corner cells
    xmin = batch.x_left_lower_corner + batch.grid_size
    xmax = xmin + (batch.Nx * batch.grid_size)
    ymin = batch.y_left_lower_corner + batch.grid_size
    ymax = ymin + (batch.Ny * batch.grid_size)

    # x coordinates of the grid cells
    xgrid = np.arange(xmin, xmax, batch.grid_size)
    # y coordinates of the grid cells
    ygrid = np.arange(ymin, ymax, batch.grid_size)

    return xgrid, ygrid

species_data = fetch_species_distributions()

lat_and_lon = np.vstack((
    species_data.train['dd lat'],
    species_data.train['dd long'],
)).T

species_train = np.array([
    d.decode('ascii').startswith('micro')  
    for d in species_data.train['species']
], dtype=np.int)

fig = plt.figure(figsize=(20, 10))  # type: plt.Figure

for i in range(2):
    ax = fig.add_subplot(1, 2, i + 1, projection=ccrs.PlateCarree())  # type: GeoAxesSubplot
    ax.set_extent([-30, -90, 15, -60], crs=ccrs.PlateCarree())  
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.RIVERS)
    ax.scatter(
        x=lat_and_lon[:, 1][species_train==0], y=lat_and_lon[:, 0][species_train==0],
        c='green' if i == 0 else 'red', cmap='rainbow', edgecolor='k', alpha=0.1
    )

x_grid, y_grid = construct_grids(species_data)
X, Y = np.meshgrid(x_grid[::5], y_grid[::5][::-1])
land_reference = species_data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()
xy = np.vstack((Y.ravel(), X.ravel())).T
xy = np.radians(xy[land_mask])

fig = plt.figure(figsize=(20, 10))  # type: plt.Figure
for i in range(2):
    ax = fig.add_subplot(1, 2, i + 1, projection=ccrs.PlateCarree())  # type: GeoAxesSubplot
    ax.set_extent([-30, -90, 15, -60], crs=ccrs.PlateCarree())  
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    ax.add_feature(cfeature.RIVERS)

    kde = KernelDensity(bandwidth=0.03, metric='haversine')  
    kde.fit(np.radians(lat_and_lon[species_train == i]))
    Z = np.full(land_mask.shape[0], -9999)
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    ax.contourf(
        X, Y, Z,
        levels=np.linspace(0, Z.max(), 50),
        cmap='Greens' if i == 0 else 'Reds',
    )
fig.savefig('GIS.png')



ModuleNotFoundError: No module named 'cartopy'